# 1. A Brief Overview for Pyramid


Show the pyramid structures: Corpus, Folder, Text, Sentence, Token

In [ ]:
import numpy as np

corpus = [['1.病史：患者为63岁女性，慢性病程，急性加重。',
'既往有“高脂血症”病史。',
'2.因“反复脐周疼痛2年余，再发并加重1周”入院。'],
['3.体查：血压128/63mmHg，神志清楚，浅表淋巴结无肿大，口唇无苍白。',
'双侧扁桃体无肿大、充血。咽无充血。颈静脉无怒张。'],
['双肺呼吸音清晰，未闻及干湿性啰音，无胸膜摩擦音。',
'心率62次/分，律齐，各瓣膜区未闻及病理性杂音。',
'腹部平坦，未见胃肠型及蠕动波，腹壁柔软，脐周压痛，无反跳痛，未扪及包块，肝脾肋下未扪及，Murphy征（-）',
'肝肾区无叩击痛，移动性浊音（-），肠鸣音4次/分。',
'双下肢无浮肿。四肢肌力、肌张力正常，生理反射存在，病理反射未引出。']]
# print(len(corpus))
corpus


In [ ]:
TEXT_DICT = {}
TEXT_DICT['NUMSents'] = []
TEXT_DICT['EndIDXSents'] = []

SENT_DICT = {}
SENT_DICT['NUMTokens'] = []
SENT_DICT['EndIDXTokens'] = []

TOKEN_DICT = {}
TOKEN_DICT['DATAToken'] = []

for text in corpus:
    # get text feature
    
    lenText = len(text)
    TEXT_DICT['NUMSents'].append(lenText)
    try:
        TEXT_DICT['EndIDXSents'].append(SENT_DICT['EndIDXTokens'][-1] + lenText)
    except:
        TEXT_DICT['EndIDXSents'].append(lenText)
    for sent in text:
        lenSent = len(sent)
        SENT_DICT['NUMTokens'].append(lenSent)
        try:
            SENT_DICT['EndIDXTokens'].append(SENT_DICT['EndIDXTokens'][-1] + lenSent)
        except:
            SENT_DICT['EndIDXTokens'].append(lenSent)
        
        
        TOKEN_DICT['DATAToken'].extend([token for token in sent])
               

print('Text  Level Dictionary')
print(TEXT_DICT)
print()
print('Sent  Level Dictionary')
print(SENT_DICT)
print()
print('Token Level Dictionary')
print(TOKEN_DICT)

In [ ]:
sentId = 0
StartIdx = SENT_DICT['EndIDXTokens'][sentId-1] if sentId != 0 else 0 # this is more faster
EndIdx   = SENT_DICT['EndIDXTokens'][sentId]

print(StartIdx, EndIdx)
print(''.join(TOKEN_DICT['DATAToken'][StartIdx: EndIdx]))
print(corpus[0][0])

# 2. Corpus to Folders and Folder to Texts

In [ ]:
import os

def CorpusFoldersReader(CORPUSPath, iden = None):
    # file is the priority
    if iden:
        corpusFiles = [i for i in os.listdir(CORPUSPath) if iden in i]
        return {os.path.join(CORPUSPath, fd): '' for fd in corpusFiles}, 'File'
    else:
        results = [x for x in os.walk(CORPUSPath) if x[2]]
        return {i[0]: i[2] for i in results},                            'Dir'
    
CORPUSPath = 'corpus/ner/'
CorpusFolders = CorpusFoldersReader(CORPUSPath, iden = None)[0]
CorpusFolders

# 3. Folder to Texts

Each text is a tuple for strText, SSET, orig_file_name, anno_file_name

In [ ]:
from nlptext.utils.pyramid import CorpusFoldersReader, FolderTextsReaders

FolderTextsReaders

In [ ]:
CORPUSPath = 'corpus/ner/'
CorpusFolders = CorpusFoldersReader(CORPUSPath, iden = None)[0]

print(CorpusFolders)

folderPath =  list(CorpusFolders.keys())[0]
fileNames  = CorpusFolders[folderPath]

textType = 'file'

TOKENLevel = 'char'
anno = '.Entity'
annoKW = {
    'sep': '\t',
    'notZeroIndex': 0,
}


FolderTexts = FolderTextsReaders[textType](folderPath, fileNames, anno, **annoKW)

strText_SSET_O_A = list(FolderTexts)[0]
    
    
strText, SSETText, origTextName, annoTextName = strText_SSET_O_A
print(strText)
print(SSETText)
print(origTextName)
print(annoTextName)

In [ ]:
CORPUSPath = 'corpus/medpos/'
CorpusFolders = CorpusFoldersReader(CORPUSPath, iden = None)[0]

# print(CorpusFolders)

folderPath =  list(CorpusFolders.keys())[0]
fileNames  = CorpusFolders[folderPath]

textType = 'file'

TOKENLevel = 'char'
anno = '.UMLSTag'
annoKW = {
    'sep': '\t',
    'notZeroIndex': 0,
}


FolderTexts = FolderTextsReaders[textType](folderPath, fileNames, anno, **annoKW)

strText_SSET_O_A = list(FolderTexts)[0]
    
    
strText, SSETText, origTextName, annoTextName = strText_SSET_O_A
print(strText)
print(SSETText)
print(origTextName)
print(annoTextName)

In [ ]:
CORPUSPath = 'corpus/boson/'
CorpusFolders = CorpusFoldersReader(CORPUSPath, iden = '.txt')[0]

# print(CorpusFolders)

folderPath =  list(CorpusFolders.keys())[0]
fileNames  = CorpusFolders[folderPath]

textType = 'line'

TOKENLevel = 'char'
anno = 'embed'

FolderTexts = FolderTextsReaders[textType](folderPath, fileNames, anno, **annoKW)

strText_SSET_O_A = list(FolderTexts)[0]
     
strText, SSETText, origTextName, annoTextName = strText_SSET_O_A
print(strText)
print(SSETText)
print(origTextName)
print(annoTextName)

The strText is then be spilted into sentences.

If SSET exists, we need to match SSET and the splited sentences.

# 4. Text to Sentences

In [ ]:
import re

##################################################################################################TEXT-SENT
def reCutText2Sent(text, useSep = False):
    
    ###################### Remove some weird chars #######################
    text = re.sub('\xa0', '', text)
    
    ############# The Issue of Spaces
    ###################### Convert the Spaces between two English Letters to 'ⴷ' #################
    # Take care of Spaces
    text = re.sub(r'(?<=[A-Za-z])\s+(?=[|A-Za-z])', 'ⴷ',  text)
    
    ###################### Convert the S+ spaces to '〰' #################
    text = re.sub(' {2}', '〰', text ).strip()
    if useSep == ' ':
        # if using space to sep the words
        text = text.replace('\t','').replace('〰', ' ')
    elif useSep == '\t':
        # if using tab to sep the words, removing all spaces
        text = text.replace(' ','').replace('〰', '')
    else:
        # if there is no sep char for Chinese, remove single space, and then convert space+ to single space
        text = text.replace('\t','').replace(' ', '',).replace('〰', ' ')
        
    # convert the spaces between English letters to single spaces
    text = text.replace('ⴷ', ' ')
    
    # Other Things
    text = re.sub('([。！!;；])([^”])', r"\1\n\2",text) 
    text = re.sub('(\.{6})([^”])',    r"\1\n\2",text) 
    text = re.sub('(\…{2})([^”])',    r"\1\n\2",text)
    
    # The \n within " " is not considered
    text = '"'.join( [ x if i % 2 == 0 else x.replace('\n', '') 
                         for i, x in enumerate(text.split('"'))] )
    text = re.sub( '\n+', '\n', text ).strip() # replace '\n+' to '\n'
    text = text.replace('\\n', '\n')
    text = text.split("\n")
    text = [sent.strip() for sent in text]
    # text = [sent.replace(' ', '').replace('\\n', '') for sent in text]
    return [sent for sent in text if len(sent)>=2]


text = '浙江在线杭州4月25日讯(记者施宇翔 通讯员 方英)毒贩很“时髦”,用微信交易毒品。没料想警方也很“潮”,将计就计,一举将其擒获。记者从杭州江干区公安分局了解到,经过一个多月的侦查工作,江干区禁毒专案组抓获吸贩毒人员5名,缴获“冰毒”400余克,毒资30000余元,扣押汽车一辆。黑龙江籍男子钱某长期落脚于宾馆、单身公寓,经常变换住址。他有一辆车,经常半夜驾车来往于杭州主城区的各大宾馆和单身公寓,并且常要活动到凌晨6、7点钟,白天则在家里呼呼大睡。钱某不寻常的特征,引起了警方注意。禁毒大队通过侦查,发现钱某实际上是在向落脚于宾馆和单身公寓的吸毒人员贩送“冰毒”。'
print(text)
reCutText2Sent(text)

In [ ]:
def segText2Sents(text, method = 'whole', **kwargs):
    
    '''
    text:
        1. textfilepath. 2. text-level string
    method: 
        1. 'whole': when text is a text-level string,then use this text-level string as sent-level string directly.
                    and return text = [sent-level string].
        2. `funct`: when method is a function, whose input is a text-level string,
                    then return text = funct(text) = [..., sent-level string, ...]
        3. 'line' : string. when text is filepath where each line is a sentence
                    then return a generator text = generate(text), item is a sent-level string.        
    '''
    if os.path.isfile(text):
        if method == 'line':
            text = lineCutText2Sent(text)
            return text
        else:
            text = fileReader(text)
    if method == 'whole':
        return [text]
    elif method == 're':
        return reCutText2Sent(text, **kwargs)
    else:
        return method(text, **kwargs)

# 5. Sentence to Tokens

In [ ]:
def segSent2Tokens(sent, method = 'iter'):
    return [i for i in sent]

# 6. Deal with Annotation

In [ ]:

strText = '''结肠多发息肉。\n患中老年男性,慢性病程。 因“体检发现大肠多发息肉3月余”入院。查体:无阳性体征。'''

print(strText)

strAnnoText = '''标注文本名称:/Users/zhangling/Documents/新标的数据530/529李选-已检查/Entity/patient4378.txt\n标注文本字数统计:87\n多发息肉\t3\t6\t疾病\n慢性\t16\t17\t修饰\n多发息肉\t30\t33\t疾病\n3月余\t34\t36\t修饰\n无阳性体征\t44\t48\t不确定\n'''
print(strAnnoText)

# BIOES

sep = '\t'
SSETText = [sset.split('\t') for sset in strAnnoText.split('\n') if sep in sset]

notZeroIndex = 1 

sset = SSETText[0]

strAnno = sset[0]
s       = int(sset[1]) - notZeroIndex
tag     = sset[3] 
CIT = [[c, s + idx, tag+ '-I']  for idx, c in enumerate(strAnno)]

CIT[-1][2] = tag + '-E'
CIT[0][2]  = tag + '-B'
    
if len(CIT) == 1:
    CIT[2] = tag + '-S'   
print(CIT)


CITAnnoText = []
for sset in SSETText:
    strAnno = sset[0]
    s       = int(sset[1]) - notZeroIndex
    tag     = sset[3] 
    CIT = [[c, s + idx, tag+ '-I']  for idx, c in enumerate(strAnno)]

    CIT[-1][2] = tag + '-E'
    CIT[0][2]  = tag + '-B'

    if len(CIT) == 1:
        CIT[0][2] = tag + '-S' 
        
    CITAnnoText.extend(CIT)
    
    
CITText = [[char, idx, 'O'] for idx, char in enumerate(strText)]

for citAnno in CITAnnoText:
    c, idx, t = citAnno
    assert CITText[idx][0] == c
    CITText[idx] = citAnno
    
CITText[:10]

`utils.getCITSents`

In [ ]:

strSents = ['结肠多发息肉。', '患中老年男性,慢性病程。']

CITText = [['结', 0, 'O'],
 ['肠', 1, 'O'],
 ['多', 2, '疾病-B'],
 ['发', 3, '疾病-I'],
 ['息', 4, '疾病-I'],
 ['肉', 5, '疾病-E'],
 ['。', 6, 'O'],
 ['\n', 7, 'O'],
 ['患', 8, 'O'],
 ['中', 9, 'O'],
 ['老', 10, 'O'],
 ['年', 11, 'O'],
 ['男', 12, 'O'],
 ['性', 13, 'O'],
 [',', 14, 'O'],
 ['慢', 15, '修饰-B'],
 ['性', 16, '修饰-E'],
 ['病', 17, 'O'],
 ['程', 18, 'O'],
 ['。', 19, 'O'],
 [' ', 20, 'O'],]



lenLastSent = 0
collapse    = 0
 
CITSents = []
for strSent in strSents:
    CITSent = []
    for sentTokenIdx, c in enumerate(strSent):
        # sentTokenIdx = txtTokenIdx - lenLastSent - collapse
        txtTokenIdx = sentTokenIdx + lenLastSent + collapse
        cT, _, tT = CITText[txtTokenIdx]
        while c != cT:
            collapse = collapse + 1
            txtTokenIdx = sentTokenIdx + lenLastSent + collapse
            cT, _, tT = CITText[txtTokenIdx]
            
        CITSent.append([c,sentTokenIdx, tT])
    lenLastSent = lenLastSent + len(strSent)
    CITSents.append(CITSent)
CITSents    

`utils.textLineReader` with `anno == 'embed'`

In [ ]:
from pprint import pprint

string = 'This is an annotated entity {{2018-12-22:date}}, try to extract it out!'

ST = [(block, 'O') if idx%2==0 else (block.split(':')[0], block.split(':')[-1]) 
    for idx, block in enumerate(string.replace("}}", '{{').split('{{'))]

pprint(ST)
# SSET, Str, S(char), E(char), Tag.

txtCharIdx = 0
SSET = []
strText = ''
for st in ST:
    string, tag = st
    sset = [string, txtCharIdx, txtCharIdx + len(string), tag]
    SSET.append(sset)
    txtCharIdx = sset[2]
    strText = strText + string
    
pprint(SSET)
pprint(strText) 

# Only Way to Check a SSET
for sset in SSET:
    assert sset[0] == strText[sset[1]: sset[2]]


# 7. Build Token String to Index Dict

In [ ]:
from nlptext.utils.infrastructure import UNK_ID, specialTokens, specialTokensDict, strQ2B, fileReader

##################################################################################################TOKEN_LTU
def buildTokens(tokenList, MaxTokenUnique = None):
    """
        Process raw inputs into a dataset.
        words: a list of the whole corpus
    """
    #########################################################################COUNT
    total_len_token = len(tokenList)
    print('The Total Number of Tokens:', total_len_token)
    print('Counting the number unique Tokens...          \t', datetime.now())
    if MaxTokenUnique:
        count = collections.Counter(tokenList).most_common(MaxTokenUnique)
    else:
        count = collections.Counter(tokenList).most_common()
    print('\t\tDone!')
    #########################################################################COUNT

    print('Generating Dictionary of Token Unique...\t', datetime.now())
    DTU = specialTokensDict.copy()
    DTU_freq = {sp_tk: 0 for sp_tk in specialTokens}
    for token, freq in count:
        if token is not specialTokens:
            DTU[token] = len(DTU)
            DTU_freq[token] = freq
        else:
            DTU_freq[token] = DTU_freq[token] + 1


    print('\t\tThe length of DTU is:', len(DTU), '\t', datetime.now())
    print('Generating the ORIGTokenIndex...       \t', datetime.now())
    data = np.zeros(len(tokenList), dtype = np.uint32)
    # data = []
    for idx, token in enumerate(tokenList):
        voc_id = DTU.get(token, UNK_ID)
        data[idx] = voc_id
        if voc_id == UNK_ID:
            DTU_freq[UNK] = DTU_freq[UNK] + 1

        # data.append(DTU.get(token,UNK_ID))
        if idx % 5000000 == 0:
            print('\t\tThe idx of token is:', idx, '\t', datetime.now())
    print('\t\tDone!')
    LTU = list(DTU.keys())

    if MaxTokenUnique:
        print('Only Keep First', MaxTokenUnique, 'Tokens.')
        print('The coverage rate is:', np.bincount(data)[UNK_ID]/total_len_token)
    # data = np.array(data)
    return data, LTU, DTU, DTU_freq
##################################################################################################TOKEN_LTU


# 8. From Corpus to Folders and From Folder to Texts


There are three methods

1. textFile

2. textLine

3. textBlock

In [ ]:
# STAGE 1
from pprint import pprint

from nlptext.utils.pyramid import CorpusFoldersReader, FolderTextsReaders

########### BOSON ###########
CORPUSPath = 'corpus/boson/'
corpusFileIden = '.txt'
textType   = 'line'
Text2SentMethod  = 're'
Sent2TokenMethod = 'iter'
TOKENLevel = 'char'
anno = 'embed'
annoKW = {}

assert anno == False or '.' in anno or anno == 'embed'
########################################################


MaxTextIdx = 10

Folders, CORPUSType = CorpusFoldersReader(CORPUSPath, iden = corpusFileIden)

pprint(Folders) # all possible files in this directory
pprint(CORPUSType)


for folderPath in Folders:
    print(folderPath)
    fileNames = Folders[folderPath]
    
    FolderTexts = FolderTextsReaders[textType](folderPath, fileNames, anno, **annoKW)
    
    for textIdx, strText_SSET_O_A in enumerate(FolderTexts):
        
        # we need to add some constraits to filter the textStrs
        strText, SSETText, origTextName, annoTextName = strText_SSET_O_A
        print(textIdx, '--', strText)
        print(SSETText, '\n')
        if textIdx == MaxTextIdx:
            break
        
        # we need to add some constraits to filter the textStrs

# 9. Text to Sentences and Sentence to Tokens

In [ ]:
# STAGE 1
from pprint import pprint
from nlptext.utils.pyramid import CorpusFoldersReader, FolderTextsReaders

# STAGE 2
from nlptext.utils.pyramid import reCutText2Sent
from nlptext.utils.pyramid import segText2Sents, segSent2Tokens# (text, method = 'whole')
from nlptext.utils.pyramid import getCITSents
from nlptext.utils.pyramid import getCITText


########### ResumeNER ###########
CORPUSPath = 'corpus/ResumeNER/'
corpusFileIden = '.bmes'
textType   = 'block'
Text2SentMethod  = 're'
Sent2TokenMethod = 'iter'
TOKENLevel = 'char'
anno = 'embed' # TODO
annoKW = {}


assert anno == False or '.' in anno or anno == 'embed'
########################################################
########################################################

MaxTextIdx = 10



########################################################
################   Things to Save   ####################
########################################################


CORPUS = {}
CORPUS['CORPUSPath'] = CORPUSPath
CORPUS['corpusFileIden'] = corpusFileIden # None if Dir else
CORPUS['CORPUSType']     = 'File' if corpusFileIden else 'Dir'
CORPUS['textType'] = textType

FOLDER = {}
FOLDER['folderPaths'] = [] 
FOLDER['NUMTexts'] = []
FOLDER['EndIDXTexts'] = []
        
TEXT = {}
TEXT['NUMSents'] = []
TEXT['EndIDXSents'] = []
TEXT['Text2SentMethod'] = Text2SentMethod
if textType == 'file':
    TEXT['ORIGFileName'] = []
if anno:
    TEXT['ANNOFileName'] = []
    
SENT = {}
SENT['NUMTokens'] = []
SENT['EndIDXTokens'] = []
SENT['Sent2TokenMethod'] = Sent2TokenMethod

TOKEN = {}
TOKEN['ORIGToken'] = []
TOKEN['TOKENLevel'] = TOKENLevel
if anno:
    TOKEN['ANNOToken'] = []

ANNO = {}
ANNO['anno'] = anno
ANNO['annoKW'] = annoKW

    
    
########################################################
##################     CHAINES      ####################
########################################################



###--> CHAIN: from Corpus to Folders <--###

CorpusFolders, CORPUSType = CorpusFoldersReader(CORPUSPath, iden = corpusFileIden)
assert CORPUS['CORPUSType'] == CORPUSType
pprint(CorpusFolders) # all possible files in this directory
pprint(CORPUSType)

for folderIdx, folderPath in enumerate(CorpusFolders):
    print(folderPath)
    fileNames = CorpusFolders[folderPath]
    
    ###--> CHAIN: from Folder to Texts <--###
    FolderTexts = FolderTextsReaders[textType](folderPath, fileNames, anno, **annoKW)
    
    for textIdx, strText_SSET_O_A in enumerate(FolderTexts):
        
        # we need to add some constraits to filter the textStrs
        strText, SSETText, origTextName, annoTextName = strText_SSET_O_A
        
        print('\n', textIdx, '--', strText)
        print(SSETText, '\n')
        
        ###--> CHAIN: from strText to strSents <--###
        strSents = segText2Sents(strText, method = Text2SentMethod) # fixed
        
        for strSent in strSents:
            #- print(strSent)
            ###--> CHAIN: from strSent to strTokens <--###
            strTokens = segSent2Tokens(strSent, method = Sent2TokenMethod)
            
            ###--> CHAIN's End: Token itself <--###
            #- print(strTokens)
            TOKEN['ORIGToken'].extend(strTokens)
            
            lenSent = len(strTokens)
            SENT['NUMTokens'].append(lenSent)
            try:
                SENT['EndIDXTokens'].append(SENT['EndIDXTokens'][-1] + lenSent)
            except:
                SENT['EndIDXTokens'].append(lenSent)
            
        
        lenText = len(strSents)
        TEXT['NUMSents'].append(lenText)
        try:
            TEXT['EndIDXSents'].append(TEXT['EndIDXSents'][-1] + lenText)
        except:
            TEXT['EndIDXSents'].append(lenText)
            
        if origTextName:
            TEXT['ORIGFileName'].append(origTextName)
            
            
        ########################################
        if anno:
            # TOKEN['ANNOToken'] = []
            # assert SSETText   != [] # May occur Errors
            for sset in SSETText:
                assert sset[0] == strText[sset[1]: sset[2]]
            if SSETText == []:
                print('\nThe SSET of this Text is Empty!!!')
                print(strText, '\n') # to check what happen
                    
            ############### PART One: Get CITText ###########
            #
            # CITText  = foo1(strText, SSETText)
            # 
            CITText = getCITText(strText, SSETText)

            
            
                
            ############### PART TWO: Get CITSents ###########
            #
            # CITSents = foo2(strSents, CITText)
            #
            CITSents = getCITSents(strSents, CITText)
           

            ############### PART THREE: Get TOKEN['ANNOToken'] ###########
            #
            # TOKEN['ANNOToken'] = foo3(CITSents, strSents)
            #
            for sentIdx, CITSent in enumerate(CITSents):
                
                # Corporate into TOKEN['ANNOToken']
                # pay attention here, CIT is char-based, but TOKEN may be word-based.
                # strTokens = segSent2Tokens(strSent, method=Sent2TokenMethod)
            
                if TOKENLevel == 'char':
                    TOKEN['ANNOToken'].extend([CITToken[2] for CITToken in CITSent])
                    #- pprint(sentIdx)
                    #- pprint(CITSent)
                else:
                    # TODO
                    pass 
            # save the file
            
            if annoTextName:
                TEXT['ANNOFileName'].append(origTextName)
            
        
        if textIdx == MaxTextIdx:
            break
    
    # Back to Folder
    lenFolder = textIdx
    FOLDER['folderPaths'].append(folderPath)
    
    FOLDER['NUMTexts'].append(lenFolder) # to remove
    try:
        FOLDER['EndIDXTexts'].append(FOLDER['EndIDXTexts'][-1] + lenFolder)
    except:
        FOLDER['EndIDXTexts'].append(lenFolder)
        
# End here
lenCorpus = folderIdx
CORPUS['NUMFolders'] = [lenCorpus]
CORPUS['EndIDXFolders'] = [lenCorpus]

# 10. Test

In [ ]:
from pprint import pprint
from nlptext.base import BasicObject

########### BOSON ###########
CORPUSPath = 'corpus/boson/'
corpusFileIden = '.txt'
textType   = 'line'
Text2SentMethod = 're'
Sent2TokenMethod = 'iter'
TOKENLevel = 'char'
anno = 'embed'
annoKW = {}


MaxTextIdx = False


BasicObject.INIT(CORPUSPath, corpusFileIden, textType, 
                 Text2SentMethod, Sent2TokenMethod, TOKENLevel, 
                 anno, annoKW, MaxTextIdx)

from nlptext.corpus import Corpus
corpus = Corpus()
# corpus.IdxFolderStartEnd

# DictToken = corpus.DictToken
# print(DictToken)

In [ ]:
print(corpus.CORPUS)

In [ ]:
print(corpus.FOLDER)

In [ ]:
print(corpus.TEXT)

In [ ]:
print(corpus.SENT)

In [ ]:
print(corpus.TOKEN)

In [ ]:

import numpy as np
txtIdxes = list(set(list(np.random.randint(corpus.TEXT['length'], size = 10))))
txtIdxes

In [ ]:
# print(corpus.Folders)
# print(corpus.FOLDER)
# print(corpus.Texts)
# print(corpus.TEXT)
# print(corpus.Sentences)
from nlptext.text import Text

sentIdx = 0
for txtIdx in txtIdxes:
    
    txt = Text(txtIdx)
    print('\n', txt, '\n')
    for st in txt.Sentences:
        print(sentIdx, '-->',st.sentence)
        sentIdx = sentIdx + 1

In [ ]:
st = corpus.Sentences[31]
st.Tokens

In [ ]:
BasicObject.TokenNum_Dir

In [ ]:
from nlptext.text import Text
loc_idx = 9  # how to interpret loc_idx: support there are n texts in the whole corpus, get the loc_idx th text
txt = Text(loc_idx)
txt.Tokens